In [1]:
import chess
import chess.pgn as pgn
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.ticker as ticker
import copy
from matplotlib.pyplot import figure
import matplotlib

# Data ETL
**Caution: If the directory "./position-analysis" existed and had data, you can pass this part.**


This part is used to extract, transform and load data from the raw chess game data in order to create a new dataset

In [2]:
players = ['Caruana', 'Polgar', 'Fischer', 'Morphy', 'Botvinnik', 'Tal',
 'Kasparov', 'Anand', 'Carlsen', 'Nakamura', 'Alekhine', 'Capablanca']

In [3]:
chess_pieces_type = [i for i in range (1, 7)]
mapp = {1: [0] * 64, 2: [0] * 64, 3: [0] * 64, 4: [0] * 64, 5: [0] * 64, 6: [0] * 64}

for player in players:
  gameList = os.listdir('../raw_game/' + player)
  for game in gameList:
    gameLink = '../raw_game/' + player + '/' + game
    
    pgn = open(gameLink)

    first_game = chess.pgn.read_game(pgn)
    board = first_game.board()
    

    for turn, move in enumerate(first_game.mainline_moves()):
      board.push(move)
      for piece in chess_pieces_type:
        li = list(board.pieces(piece_type=piece, color=True))
        # print(li)
        for i in li:
          mapp[piece][i] += 1

white_data = pd.DataFrame(mapp)
white_data.rename(columns = {1:'P', 2:'N', 3:'B', 4:'R', 5:'Q', 6:'K'}, inplace=True)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../raw_game/Caruana'

: 

In [10]:
chess_pieces_type = [i for i in range (1, 7)]
mapp = {1: [0] * 64, 2: [0] * 64, 3: [0] * 64, 4: [0] * 64, 5: [0] * 64, 6: [0] * 64}

for player in players:
  gameList = os.listdir('../raw_game/' + player)
  for game in gameList:
    gameLink = '../raw_game/' + player + '/' + game
    
    pgn = open(gameLink)

    first_game = chess.pgn.read_game(pgn)
    board = first_game.board()
    

    for turn, move in enumerate(first_game.mainline_moves()):
      board.push(move)
      for piece in chess_pieces_type:
        li = list(board.pieces(piece_type=piece, color=False))
        # print(li)
        for i in li:
          mapp[piece][i] += 1

black_data = pd.DataFrame(mapp)
black_data.rename(columns = {1:'p', 2:'n', 3:'b', 4:'r', 5:'q', 6:'k'}, inplace=True)

FileNotFoundError: [WinError 3] The system cannot find the path specified: '../raw_game/Caruana'

In [ ]:
piece_data = white_data.join(black_data)
piece_data.insert(loc=0, column = 'Square', value = range(0, len(piece_data)))
piece_data.to_csv('position-analysis.csv', index=False)

NameError: name 'white_data' is not defined

# Data EDA

In [ ]:
piece_data = pd.read_csv("position-analysis.csv")

NameError: name 'pd' is not defined

In [1]:
piece_data.describe()

NameError: name 'piece_data' is not defined

In [ ]:
chess_pieces = ['P', 'R', 'N', 'B', 'Q', 'K', 
            'p', 'r', 'n', 'b', 'q', 'k']

columns = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
rows = [i for i in range(1, 9)]

sumOfChessPostion = piece_data.sum(axis=0)

# Calculate the probability of each postion with each piece
for piece in chess_pieces:
  piece_data[piece] = piece_data[piece] / sumOfChessPostion[piece]

# Transform sqare index to the actual position in board in order to make visual
piece_data['Row'] = piece_data['Square'].apply(
  lambda x: rows[int(x / 8)]
)

piece_data['Column'] = piece_data['Square'].apply(
  lambda x: columns[x % 8]
)


NameError: name 'piece_data' is not defined

In [1]:
piece_data

NameError: name 'piece_data' is not defined

In [ ]:
norm = colors.LogNorm(vmin=1e-4, vmax=1e-1) 
cmap = copy.copy(matplotlib.cm.get_cmap('coolwarm'))
cmap.set_bad(cmap(0))
titleOfPiece = {'p': 'Pawn', 'n': 'Knight', 'b': 'Bishop', 'r': 'Rook', 'q': 'Queen', 'k': 'King'}

fig, ax = plt.subplots(4, 3)
fig.set_figheight(15)
fig.set_figwidth(15)

iter = 0
for key in chess_pieces:
  this_ax = ax[np.unravel_index(iter, (4, 3))]
  iter += 1

  color = 'White' if (key.isupper()) else 'Black'
  title = color + ' ' + titleOfPiece[key.lower()]

  iloc = piece_data.columns.get_loc(key)
  data = piece_data.iloc[::-1,  iloc].to_numpy().reshape((8, 8))

  im = this_ax.imshow(data, norm = norm, cmap = cmap)
  
  this_ax.xaxis.set_major_formatter(ticker.NullFormatter())
  this_ax.xaxis.set_major_locator(ticker.FixedLocator([-0.5, 0.5,1.5,2.5,3.5,4.5,5.5, 6.5]))
  this_ax.xaxis.set_minor_locator(ticker.FixedLocator([0, 1, 2, 3, 4, 5, 6, 7]))
  this_ax.xaxis.set_minor_formatter(ticker.FixedFormatter(['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']))

  this_ax.yaxis.set_major_formatter(ticker.NullFormatter())
  this_ax.yaxis.set_major_locator(ticker.FixedLocator([-0.5, 0.5,1.5,2.5,3.5,4.5,5.5, 6.5]))
  this_ax.yaxis.set_minor_locator(ticker.FixedLocator([0, 1, 2, 3, 4, 5, 6, 7]))
  this_ax.yaxis.set_minor_formatter(ticker.FixedFormatter(['8', '7', '6', '5', '4', '3', '2', '1']))

  this_ax.set_title(title)
  this_ax.grid(color = 'black')


cbar_ax = fig.add_axes([0.9, 0.15, 0.05, 0.7])
cbar = fig.colorbar(im, extend = 'both', cax=cbar_ax)

fig.suptitle('Probability of Chess Piece Positions', fontsize = 24)

plt.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=0.9, 
                    hspace=0.3)

fig.patch.set_facecolor('white')
plt.savefig('../analysis/probability-chess-position.png', transparent=False)
plt.show()


Viet phan tich cho hinh tren nha

>Các quân tốt ở giữa có xu hướng tham gia kiểm soát trung tâm, đặc biệt là vị trí d4, e4 với tốt trắng và d5, e5 với tốt đen, các quân tốt còn lại không di chuyển nhiều để tạo hàng rào bảo vệ tướng.

> Xe tập trung di chuyển ở hàng 1 đối với xe trắng và hàng 8 đối với xe đen do khả năng di chuyển của xe.

> Mã tham gia vào chiếm lĩnh trung tâm cùng với các tốt ở giữa, hai vị trí yêu thích của mã là c3, f3 đối với mã trắng và c6, f6 đối với mã đen

> Tịnh tập trung chiếm lĩnh các đường chéo lớn do khả năng di chuyển của tịnh.

>Hậu di chuyển nhiều và đa dạng trên bàn cờ.

> Vua hầu như đứng yên trong cả trận đấu, vua có hai vị trí yêu thích là vị trí ban đầu và vị trí sau khi nhập thành cánh xe.
